In [1]:
require 'daru'
require 'daru_plotting_plotly'
require 'rbplotly'
require 'hg'

class Numeric
  def mili
    self.to_f * 10**(-3)
  end
end

Daru.plotting_library = :plotly

:plotly

# 実験5

In [2]:
['0', '75', '150'].select { |name| File.exists? "data/jikken5_#{name}.csv" }.map do |name|
  h = Daru::DataFrame.from_csv("data/jikken5_#{name}.csv").to_h
  [
    {x: h['Ei'].to_a, y: h['Eo'].to_a, name: "Io=#{name}mA"},
    {x: h['Ei'].to_a, y: h['Vz'].to_a, name: "Io=#{name}mA"}
  ]
end.tap do |arr|
  arr.transpose.zip(['Eo', 'Vz']).each do |trace, kind|
    Plotly::Plot.new(data: trace, layout: { xaxis: { title: 'Ei(V)'}, yaxis: { title: "#{kind}(V)" }, title: kind }).show
  end
end
nil

#<Plotly::Offline::HTML:0x007fd89100a6d8 @id="29d4cf4d-4b87-4ce1-b95a-970db3a4c914", @data=[{:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[9.4, 9.5, 9.53, 9.58, 9.65, 9.62, 9.65, 9.69, 9.7], :name=>"Io=0mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[8.23, 9.2, 9.4, 9.5, 9.52, 9.58, 9.6, 9.61, 9.65], :name=>"Io=75mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[6.82, 7.8, 8.5, 9.54, 9.5, 9.55, 9.6, 9.62, 9.64], :name=>"Io=150mA"}], @layout={:xaxis=>{:title=>"Ei(V)"}, :yaxis=>{:title=>"Eo(V)"}, :title=>"Eo"}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fd892880c80 @id="13f66e2c-b2cc-4297-b84a-5351df450a9d", @data=[{:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[5.56, 5.58, 5.59, 5.59, 5.6, 5.6, 5.61, 5.61, 5.61], :name=>"Io=0mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[5.54, 5.56, 5.57, 5.58, 5.59, 5.59, 5.6, 5.6, 5.61], :name=>"Io=75mA"}, {:x=>[10, 11, 12, 13, 14, 15, 16, 17, 18], :y=>[5.49, 5.53, 5.55, 5.57, 5.58, 5.59, 5.6, 5.6, 5.61], :name=>"Io=150mA"}], @layout={:xaxis=>{:title=>"Ei(V)"}, :yaxis=>{:title=>"Vz(V)"}, :title=>"Vz"}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# 実験6

In [3]:
Daru::DataFrame.from_csv('data/jikken6.csv').plot(
  x: 'Io', y: 'Vo',
  layout: {
    title: '出力電圧',
    yaxis: { title: '出力電圧(V)' },
    xaxis: { title: '出力電流(A)' }
    }
  ).show
Daru::DataFrame.from_csv('data/jikken6.csv').plot(
  x: 'Io', y: 'ripple_ratio',
  layout: {
    title: 'リップル含有率',
    yaxis: { title: 'リップル含有率(％)' },
    xaxis: { title: '出力電流(A)' }
    }
  ).show

#<Plotly::Offline::HTML:0x007fd891b77c28 @id="730abd85-0136-47cf-ba30-c9a5bebfed32", @data=[{:x=>[0.002, 0.022, 0.042, 0.06, 0.08, 0.098, 0.12, 0.14, 0.158, 0.18, 0.2], :y=>[9.72, 9.7, 9.69, 9.65, 9.62, 9.6, 9.51, 9.34, 9.13, 8.82, 8.65], :name=>"Vo", :type=>:scatter, :mode=>""}], @layout={:xaxis=>{:title=>"出力電流(A)"}, :yaxis=>{:title=>"出力電圧(V)"}, :title=>"出力電圧", :width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fd8910feff8 @id="27e13718-03a7-45a1-a48b-4535a72d5627", @data=[{:x=>[0.002, 0.022, 0.042, 0.06, 0.08, 0.098, 0.12, 0.14, 0.158, 0.18, 0.2], :y=>[0.24691358024691357, 0.577319587628866, 0.9081527347781218, 1.2435233160621761, 1.7151767151767152, 2.447916666666667, 9.674027339642482, 20.342612419700217, 28.477546549835704, 37.41496598639456, 46.24277456647399], :name=>"ripple_ratio", :type=>:scatter, :mode=>""}], @layout={:xaxis=>{:title=>"出力電流(A)"}, :yaxis=>{:title=>"リップル含有率(％)"}, :title=>"リップル含有率", :width=>1000, :height=>500}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

# 実験7

In [4]:
['0', '75', '150'].select { |name| File.exists? "data/jikken7_#{name}.csv" }.map do |name|
  h = Daru::DataFrame.from_csv("data/jikken7_#{name}.csv").to_h
  {x: h['Vi'].to_a, y: h['Vo'].to_a, name: "Io=#{name}mA"}
end.tap do |trace|
  Plotly::Plot.new(data: trace, layout: { xaxis: { title: 'Es(V)'}, yaxis: { title: 'Eo(V)' } }).show
end
nil

#<Plotly::Offline::HTML:0x007fd892325c90 @id="ea9331f3-206d-4345-ab75-bcbf1797ff63", @data=[{:x=>[13.01, 14.03, 15.01, 16.01, 17.0], :y=>[9.67, 9.7, 9.72, 9.76, 9.79], :name=>"Io=0mA"}, {:x=>[13, 14.01, 15.0, 15.99, 17.0], :y=>[9.55, 9.6, 9.64, 9.69, 9.71], :name=>"Io=75mA"}, {:x=>[12.99, 14.03, 15.01, 16.04, 17.03], :y=>[8.59, 8.99, 9.25, 9.48, 9.62], :name=>"Io=150mA"}], @layout={:xaxis=>{:title=>"Es(V)"}, :yaxis=>{:title=>"Eo(V)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

In [4]:
r3 = 3.3 * 1000
r4 = 6.3 * 1000

5.6/(r4/(r3+r4))

8.533333333333333

In [2]:
r1 = 1.33 * 1000
r2 = 1.55 * 1000

r4*r3/(r4+r3)

2165.625